Домашнее задание #2. Парсинг

In [25]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

**Задача** - собрать данные по всем текущим преподавателям всех `институтских кафедр` МФТИ.

Собранный датасет должен представлять из себя `.csv` файл со следующими полями:

* `full_name` - ФИО преподавателя
* `birth_day` - Дата рождения преподавателя
*  `teach_place` - Кафедра
*  `degree`- Ученая Степень
* `knowledge` - Знания
*  `teaching_skill` - Умение преподавать
* `commication_skill` - В общении
* `easy_exam` - «Халявность»
* `overall_score` - Общая оценка



0. Открываем с помощью Selenium клавную страницу вики-Физтех- http://wikimipt.org/wiki/

In [ ]:
path_CDM = 'data/chromedriver'
url = 'http://wikimipt.org/wiki/'

options = Options()
options.headless = True
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)


# 1. Получаем список кафедр с главной страницу вики-Физтех

titles = browser.find_elements_by_xpath('//*[@class="mw-headline"]')
faculties = []
# ищем Институтские кафедры, потом его родителя должно быть <h3>, затем брата div, и его список - это и есть кафедры
for i in titles:
    if i.text == 'Институтские кафедры':
        faculties = i.find_elements_by_xpath('../following-sibling::div/ul/li/a')

# берем из каждой кафедры ссылку и атрибут class - он нам нужен будет для проверки существования страницы кафедры
faculties = [{'url': f.get_attribute('href'), 'cls': f.get_attribute('class')} for f in faculties]

# Обходя список кафедр, "проваливаемся" в каждую страницу кафедру кликом по названию кафедры
# реализовать это у меня не получилось - я натыкался на ошибку element is not attached to page document
# (StaleElementReferenceException), поэтому пришлось сохранять ссылки на кафедры, а затем преподавателей в массив...

# 2. Получаем список преподавателей
all_teachers = []

for f in faculties:
    if f.get('cls') == 'new':
        continue
    browser.get(f.get('url'))
    titles = browser.find_elements_by_xpath('//*[@class="mw-headline"]')
    title_teachers = None
    for t in titles:
        if t.text == 'Преподаватели кафедры':
            title_teachers = t
    if not title_teachers:
        print(f'Department: {f.get("url")} has no teacher title')
        continue
    teachers = title_teachers.find_elements_by_xpath('(../following-sibling::div)[1]/ul//a')
    all_teachers += [t.get_attribute('href') for t in teachers]


teacher_info = {
    'full_name': None,
    'birth_day': None,
    'teach_place': None,
    'degree': None,
    'knowledge': None,
    'teaching_skill': None,
    'communication_skill': None,
    'easy_exam': None,
    'overall_score': None
}
dataset = {k: [] for k, _ in teacher_info.items()}


def update_teacher_info(title, key):
    t = title.find_element_by_xpath('following-sibling::td//*[@class="starrating-avg"]').text
    try:
        teacher_info[key] = float(t.split()[0])
    except (IndexError, AttributeError, ValueError):
        print(f'Teacher {teacher_info.get("full_name")} has no rate {key}')

# Обходя список преподавателей "Проваливаемся" на страницу преподавателя по ФИО преподавателя
# 3. Получаем табличку данных с характеристиками преподавателями и прочую необходимую информацию.

for t in all_teachers:
    browser.get(t)

    teacher_info['full_name'] = browser.find_element_by_id("firstHeading").text
    brt_dgr_wrk = browser.find_elements_by_xpath('//*[@class="wikitable card"]//th')
    for t in brt_dgr_wrk:

        if t.text == 'Дата рождения':
            birthday = t.find_element_by_xpath('following-sibling::td').text
            teacher_info['birth_day'] = birthday
        if t.text == 'Учёная степень':
            degree = t.find_element_by_xpath('following-sibling::td').text
            teacher_info['degree'] = degree.rsplit(' ', 1)[0]
        if t.text == 'Работает':
            work = t.find_element_by_xpath('following-sibling::td').text
            teacher_info['teach_place'] = work

    other = browser.find_elements_by_xpath('//*[@class="wikitable card"]//table//td')

    for o in other:
        if o.text == 'Знания':
            update_teacher_info(o, 'knowledge')
        if o.text == 'Умение преподавать':
            update_teacher_info(o, 'teaching_skill')
        if o.text == 'В общении':
            update_teacher_info(o, 'communication_skill')
        if o.text == '«Халявность»':
            update_teacher_info(o, 'easy_exam')
        if o.text == 'Общая оценка':
            update_teacher_info(o, 'overall_score')

    for k, v in teacher_info.items():
        dataset[k].append(v)

    # browser.quit()
browser.quit()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Skygen\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Skygen\AppData\Local\Temp/ipykernel_5396/2681334142.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
C:\Users\Skygen\AppData\Local\Temp/ipykernel_5396/2681334142.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  titles = browser.find_elements_by_xpath('//*[@class="mw-headline"]')
d:\mipt\mipt-datascience\venv\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Us

Teacher Балабанов Юрий Вячеславович has no rate knowledge
Teacher Балабанов Юрий Вячеславович has no rate teaching_skill
Teacher Балабанов Юрий Вячеславович has no rate communication_skill
Teacher Балабанов Юрий Вячеславович has no rate easy_exam
Teacher Балабанов Юрий Вячеславович has no rate overall_score
Teacher Гарбер Алексей Игоревич has no rate teaching_skill
Teacher Гарбер Алексей Игоревич has no rate communication_skill
Teacher Гриневич Алексей Иванович has no rate knowledge
Teacher Гриневич Алексей Иванович has no rate teaching_skill
Teacher Гриневич Алексей Иванович has no rate communication_skill
Teacher Гриневич Алексей Иванович has no rate easy_exam
Teacher Гриневич Алексей Иванович has no rate overall_score
Teacher Гусятников Пётр Борисович has no rate knowledge
Teacher Гусятников Пётр Борисович has no rate teaching_skill
Teacher Гусятников Пётр Борисович has no rate communication_skill
Teacher Гусятников Пётр Борисович has no rate easy_exam
Teacher Гусятников Пётр Борисо

Сохраняем данные в pandas DataFrame

In [94]:
df = pd.DataFrame.from_dict(dataset)

# сохраняем в csv
df.to_csv(r'data/mipt_teachers.csv', header=True, sep=';', index=False)

                                          full_name        birth_day  \
0                      Агаханов Назар Хангельдыевич     25 июня 1954   
1                           Агаханова Яна Сергеевна     25 июня 1954   
2                        Алексеев Василий Антонович     25 июня 1954   
3                       Амосов Григорий Геннадьевич   27 ноября 1972   
4                            Андреев Михаил Юрьевич   27 ноября 1972   
...                                             ...              ...   
1512  Файл:Скворчевский Константин Анатольевич.jpeg  5 сентября 1929   
1513  Файл:Скворчевский Константин Анатольевич.jpeg  5 сентября 1929   
1514            Скворчевский Константин Анатольевич  5 сентября 1929   
1515           Файл:Бубнов Григорий Георгиевич.jpeg  5 сентября 1929   
1516                     Бубнов Григорий Георгиевич  5 сентября 1929   

                                         teach_place  \
0                          Кафедра высшей математики   
1                      